## Modeling

Python version spiral ... 
- > tf 2.11.0 introduced changes that break recommenders integration, need to step back in time 
- pyenv install 3.9
- pyenv virtualenv 3.9 recommenders
- pyenv uninstall recommenders
- pyenv activate recommenders

In [ ]:
%set_env TF_USE_LEGACY_KERAS=True

: 

In [4]:
import pandas as pd
import keras 

2025-07-20 21:53:46.708500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-20 21:53:46.753473: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-20 21:53:46.755234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-20 21:53:46.755243: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [5]:
reviews = pd.read_parquet("../data/2023/reviews_small.parquet")
items = pd.read_parquet("../data/2023/items_small.parquet")

In [6]:
reviews

,rating,parent_asin,user_id,timestamp
3556,4,B07XXWZKBT,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:39:36.234
3557,5,B0BRZFGB6R,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:38:29.878
3558,3,B09VBWBHBC,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:22:03.994
3559,5,B09GLQ2PFV,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-05-07 21:54:28.387
3560,3,B084HM5WG1,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-03-06 17:53:00.543
...,...,...,...,...
9995511,5,B00CLFB3ZI,AEJVQFZNE5CH5TFE2EFOXSQ5LX6Q,2016-04-07 15:58:10.000
9995512,5,B003C8RGJ0,AEJVQFZNE5CH5TFE2EFOXSQ5LX6Q,2015-04-22 17:15:34.000
9995513,4,B00B91KYV4,AEJVQFZNE5CH5TFE2EFOXSQ5LX6Q,2014-11-19 01:01:47.000
9995514,5,B004NDEWWW,AEJVQFZNE5CH5TFE2EFOXSQ5LX6Q,2014-09-05 02:12:45.000


In [7]:
items

,title,average_rating,rating_number,description,price,images,parent_asin
102,KHOMO - iPad 2 3 and 4 Generation Case - DUAL ...,4.5,2745,[KHOMO dual Series case choom introduces its b...,11.95,[{'hi_res': 'https://m.media-amazon.com/images...,B06XKRXLDR
140,HGST Touro Desk Pro 3TB USB 3.0 External Hard ...,3.2,15,"[Product Description, Hitachi Touro Desk Pro 3...",None,[{'hi_res': 'https://m.media-amazon.com/images...,B004W7DR02
153,Tiffen 77mm Photo Essentials Kit with UV Prote...,4.6,576,"[Product Description, This kit consists of a: ...",89.99,"[{'hi_res': None, 'large': 'https://m.media-am...",B0067HY1EQ
190,"Technical Pro C-QS-1225 25' 1/4"" to Speakon Sp...",4.4,11,[Available in 12 and 16 gauge speaker wire. Av...,15.95,[{'hi_res': 'https://m.media-amazon.com/images...,B0029L7IYY
234,CaseBot Leather Case for Kindle Oasis (10th an...,4.5,1444,"[CaseBot, a premium series brought to you by F...",7.99,[{'hi_res': 'https://m.media-amazon.com/images...,B07M67FJDB
...,...,...,...,...,...,...,...
1380025,MONICONA Air Tube Earpiece Surveillance Kit Ea...,4.0,35,[],29.99,[{'hi_res': 'https://m.media-amazon.com/images...,B09N8YFX24
1380045,"Galaxy Tab S2 8.0 Case, IVSO Samsung Galaxy Ta...",3.7,62,[],None,"[{'hi_res': None, 'large': 'https://m.media-am...",B00UP5Q67Q
1380090,Drop Protected Case Cover Compatible with Airp...,4.1,426,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B08CBV77FD
1380577,Logitech iPad Pro 10.5 inch Keyboard Case | SL...,4.3,1581,[Slim Combo is the ultimate companion to your ...,None,[{'hi_res': 'https://m.media-amazon.com/images...,B0753223KN


We need to build a compact user-centric representation of preference, collapse review data into a sparse matrix of user -> item preferences. There are some heuristics that need to be applied in the process: 
1. users with few interactions are a very weak signal -- without associations with multiple products, we are not teaching the model about positive associations
2. products with few interactions are also a very weak signal -- we are looking to connect users and items that have tiny interaction graphs are not going to improve our macro-level predictions

In [8]:
# TODO: This should be a configuration option hyper parameter, we can relax this if training isn't suepr computationally expensive
min_ratings = 50
items = items[items.rating_number > min_ratings]
len(items) 

13739

In [9]:
# The above item filtering serves to reduce the computational complexity as well as 
# reduce sparsity, before we filter reviews make sure we remove those associated with 
# dropped items
all_items = set(items.parent_asin)
reviews = reviews[reviews.parent_asin.isin(all_items)]
reviews.rename(columns={'parent_asin':'item_id'}, inplace=True)

/tmp/ipykernel_705786/1979523271.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.rename(columns={'parent_asin':'item_id'}, inplace=True)


In [10]:
users = reviews.groupby(['user_id']).rating.count()
users = pd.DataFrame(users).reset_index()
users.rename(columns={'rating':'ratings'}, inplace=True)

In [11]:
# This is a configuration parameter, as above ... relax if we don't have issues with compute
min_reviews = 10
users = users[users.ratings > min_reviews] 
users

,user_id,ratings
0,AE2254CLKQFGLVCWHFRAMSW4CNZQ,21
1,AE25DZNJUA4LF4CLKKF4QPT4FGTQ,38
2,AE25GLEND5ZFPDAQGUQYLMOZO6OQ,47
3,AE27EIR4TWXNPPU7T2BEY6KLOZPA,27
6,AE2PYLWJATZRIBVWEPWEAWB6DHDQ,16
...,...,...
995,AHZH6HT3H7BJWXLHDXEDLBNVESDQ,24
996,AHZJI67Q35ZP6VYLCYPZNBY4EI6A,18
997,AHZRDAQ2DUMUE43OEIKOIIPJMU2Q,31
998,AHZVRZG2OPZCTDQCH5RWZL6AGKCA,12


❗in the notebook, ratings are thresholded ... do we need to follow suit? what are the ramifications if we don't? OH... in the notebook, a click is an interaction, there's no middle ground. the network is going to operate on 0s or 1s. by leaving low reviews in our matrix, the network would learn to recommend things users have interacted with, but not necessarily positively. our case is the same, a review is an interaction. we're aiming to recommend, and we should not want to recommend low reviews. so filter... 

In [13]:
# Discard reviews by users outside our core group 
reviews = reviews[reviews.user_id.isin(set(users.user_id))]

In [14]:
reviews

,rating,item_id,user_id,timestamp
3556,4,B07XXWZKBT,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:39:36.234
3557,5,B0BRZFGB6R,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:38:29.878
3558,3,B09VBWBHBC,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:22:03.994
3559,5,B09GLQ2PFV,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-05-07 21:54:28.387
3560,3,B084HM5WG1,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-03-06 17:53:00.543
...,...,...,...,...
9982314,3,B00E1CRC86,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:32:41.000
9982316,3,B004ZMG55I,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:32:00.000
9982317,4,B0033Z2BAQ,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:31:52.000
9982318,4,B019SU6YA2,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:31:47.000


In [16]:
import numpy as np 
import pandas as pd 

### Naive

### Nearest Neighbor

In [ ]:
# If we need sparse types, use the scipy COO since it seems to be incorporated in both pytorch and recommenders
from scipy import coo_matrix

In [58]:
from recommenders.datasets.sparse import AffinityMatrix

In [ ]:
reviews = reviews.drop(['timestamp'])

In [123]:
header = {
    "col_user": "user_id",
    "col_item": "item_id",
    "col_rating": "rating",
}
ui_sparse = AffinityMatrix(reviews, **header)

# This isn't implied by the name, but this densifies the matrix, i.e. we have a contiguous u x i
# matrix here (user vector of item ratings) ... though it's actually not clear how the memory is 
# managed underneath in scipy, the 'dense' array might just be a bunch of pointers to the DFs stored 
# in the AM object... 
ui_dense, u_map, i_map = ui_sparse.gen_affinity_matrix()

In [92]:
import sys 
print(sys.getsizeof(ui_sparse))
print(sys.getsizeof(ui_sparse.df))
print(sys.getsizeof(ui_dense))

48
2608160
128


In [131]:
u_map

{'AE2254CLKQFGLVCWHFRAMSW4CNZQ': 0,
 'AE25DZNJUA4LF4CLKKF4QPT4FGTQ': 1,
 'AE25GLEND5ZFPDAQGUQYLMOZO6OQ': 2,
 'AE27EIR4TWXNPPU7T2BEY6KLOZPA': 3,
 'AE2PYLWJATZRIBVWEPWEAWB6DHDQ': 4,
 'AE2UU6RXXJ2UIJBXVVX65BKH2KPA': 5,
 'AE346DRDWBR4GQ7WELOVEJCWFPBQ': 6,
 'AE3DPJUSETWEPBSPUSWZJ7MXVC7A': 7,
 'AE3EEXRRH7HKXOKNCMAO5GRPLTSQ': 8,
 'AE3G2CDTHMYALSSQMLFE4DDS7OCA': 9,
 'AE3I336SJDH7DEBHIB6KFBZYNB3A': 10,
 'AE3RA7AB5MZUMGRPGHJWZHMOHYIQ': 11,
 'AE425ZMRX3VOQQCIGBKGWU3SUOFQ': 12,
 'AE4CJ6OTFLXXUWXIQAG3SEWQZIPA': 13,
 'AE4CPSCYTZKUQLZHF5PP5MXD6FIA': 14,
 'AE4LV63B4X2JTC55F7OBYHUPV66Q': 15,
 'AE4OXRV7XYBYAS26IH6RG354X7SQ': 16,
 'AE4UTOZX7VTWV3F6QP6NPL35ZGNA': 17,
 'AE4XFXKXW5ZHXRBSEHM6YFJWKVZA': 18,
 'AE5EFG5RI3Z3YSUJTSV7DPZB6BCQ': 19,
 'AE5LUXJBQTLBIK7YM2U4FFYJPQWQ': 20,
 'AE5PVJVGRTC4IQIIIXDRTD6ZNS7Q': 21,
 'AE5SEY3TVSWSP6ZSWV2R5GAEEALQ': 22,
 'AE5SIKE4HJT4BGJHMSTP5PQGQRAA': 23,
 'AE5UPXGAYV3H7W5QOMDXK6WG3EFA': 24,
 'AE5VGWADPKGWDCGTNDXMXYINLHJQ': 25,
 'AE5YEJQPNE57GQDYO4J5T5N6UO7Q': 26,
 'AE5ZEKWXI

In [125]:
i_map

{'B077NJ1GFS': 0,
 'B07987KN5V': 1,
 'B09WKJQKGJ': 2,
 'B07C54DVHR': 3,
 'B074X1DJLY': 4,
 'B011BRUOMO': 5,
 'B08QW5WT92': 6,
 'B06W2PMP6C': 7,
 'B01BA6XQI0': 8,
 'B07TG8JNJT': 9,
 'B00LP6CFEC': 10,
 'B09SHLTSG7': 11,
 'B079GPFLT1': 12,
 'B07BRHB8C1': 13,
 'B0B2W22XHB': 14,
 'B099W77VPG': 15,
 'B01L8GG6PW': 16,
 'B0BS2TMK39': 17,
 'B097STBFF9': 18,
 'B07PDGLWG6': 19,
 'B07F2RTN5Z': 20,
 'B0BJB7XNLG': 21,
 'B07CJYMRWM': 22,
 'B00PFQPX5Q': 23,
 'B08GL19L4T': 24,
 'B01LWKU27Z': 25,
 'B075X8471B': 26,
 'B000H2W270': 27,
 'B01K8B8YA8': 28,
 'B01GW7YJTC': 29,
 'B00B8KXLT8': 30,
 'B074KCG6LJ': 31,
 'B09XV625Z1': 32,
 'B01DBEFRU4': 33,
 'B00EZMZULI': 34,
 'B0CH9HZG73': 35,
 'B07CBYHCL9': 36,
 'B087QFQW6F': 37,
 'B07822JQ43': 38,
 'B08F1P3BCC': 39,
 'B089F9YDFC': 40,
 'B0BVJ6GG1V': 41,
 'B0C1VP1GGK': 42,
 'B08KYCZVMJ': 43,
 'B0B4ZV9WF9': 44,
 'B07KTYJ769': 45,
 'B0854DFHBH': 46,
 'B07KK62GB7': 47,
 'B014FEM0X6': 48,
 'B07MLY3JKV': 49,
 'B07NQSQRWV': 50,
 'B07VTK654B': 51,
 'B07HZLHPKP': 52,
 'B

In [150]:
ui_dense[0] 

array([5, 5, 5, ..., 0, 0, 0])

In [168]:
ui_dense[1][[0,413]]

array([0, 0])

In [185]:
a = np.nonzero(ui_dense[0])[0]
b = np.nonzero(ui_dense[1])[0]

In [186]:
np.concatenate([a, b], axis=0) 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57])

In [ ]:
ui_dense[0][np.nonzero(ui_dense[0])]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
sim_a = cosine_similarity([ui_dense[0]], [ui_dense[33]])
sims = [0] *len(users)

In [ ]:
sims[0] = sim_a

In [200]:
top_k = 5

In [205]:
[[0]*2]*top_k 

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

/tmp/ipykernel_705786/1659084739.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([[0]*3, [0]*top_k])


array([list([0, 0, 0]), list([0, 0, 0, 0, 0])], dtype=object)

In [ ]:
# Populating a full user similarity matrix is inherently limited by (and is a questionable 
# strategy because of) the u^2 memory requirement. Unlike our affinity matrices, 
# these are not sparse. Since our goal is recommending items, we'll compute the similarity
# iterativealy and store the most similar users to get down to C * u memory
similarity_matrix = np.array([[0.] * top_k] * len(users))
for a in range(len(users)): 

    # Collect our similarities w/ respect to user A
    sim_a = {}
    for b in range(len(users)): 
        if a != b: 
            
            # Given the sparsity of our review vectors, cosine similarity is going to be 
            # effectively zero if we look across the entire item space... compare only those 
            # items these two users have in common (at least 1 rating between the two)
            a_item_ix = np.nonzero(ui_dense[a])[0]            
            b_item_ix = np.nonzero(ui_dense[b])[0]
            all_ix = np.concatenate([a_item_ix, b_item_ix])
            a_items = ui_dense[a][all_ix]
            b_items = ui_dense[b][all_ix]
            sim_a[b] = cosine_similarity([a_items], [b_items])[0][0]
    
    # Find and store the top k user matches, in order    
    # NOTE: dict sorting logic courtesy of gpt-4o (https://chatgpt.com/share/687dc72f-54b4-8013-806e-b1de20d0ef12)
    top = sorted(sim_a.items(), key=lambda x: x[1], reverse=True)[:top_k]
    similarity_matrix[a] = [x[0] for x in top]



In [260]:
similarity_matrix

array([[606., 742., 236., 730.,  95.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.]])

### VAE

In [ ]:
from recommenders.datasets.sparse import AffinityMatrix

In [139]:
# Only relevant to VAE strategy 
reviews = reviews[reviews.rating >= 3]

In [27]:
reviews.columns

Index(['rating', 'item_id', 'user_id', 'timestamp'], dtype='object')

In [28]:
reviews

,rating,item_id,user_id,timestamp
3556,4,B07XXWZKBT,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:39:36.234
3557,5,B0BRZFGB6R,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:38:29.878
3558,3,B09VBWBHBC,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-07-17 02:22:03.994
3559,5,B09GLQ2PFV,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-05-07 21:54:28.387
3560,3,B084HM5WG1,AHBZRDFYB2FWUAO63DCSF2VSTJ2Q,2022-03-06 17:53:00.543
...,...,...,...,...
9982314,3,B00E1CRC86,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:32:41.000
9982316,3,B004ZMG55I,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:32:00.000
9982317,4,B0033Z2BAQ,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:31:52.000
9982318,4,B019SU6YA2,AFCFZWIEVL4RIJM66PUTOYA4LFIQ,2016-03-09 01:31:47.000


In [29]:
# NOTE: Strategy adapted from tutorials available in the Recommenders project, see 
# https://github.com/recommenders-team/recommenders/tree/main
from recommenders.datasets.python_splitters import python_random_split

# Split along user boundaries to ensure no leakage of preference between train and test
train_users, test_users, val_users = python_random_split(users, [.9, .05, .05])

In [30]:
print(train_users.shape, test_users.shape, val_users.shape)

(734, 2) (40, 2) (41, 2)


In [31]:
train = reviews[reviews.user_id.isin(train_users.user_id)]
val = reviews[reviews.user_id.isin(val_users.user_id)]
test = reviews[reviews.user_id.isin(test_users.user_id)]

In [32]:
print(train.shape, val.shape, test.shape)

(13241, 4) (845, 4) (733, 4)


In [33]:
# Technique from Recommenders (see https://github.com/recommenders-team/recommenders/blob/45e1b215a35e69b92390e16eb818d4528d0a33a2/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) 
# to improve utility of validation set during training - only allow items in
# the validation set that are also present in the train set
val = val[val.item_id.isin(train.item_id.unique())]

In [34]:
val.shape

(272, 4)

In [35]:
from recommenders.datasets.python_splitters import python_stratified_split 

# Another technique employed in Recommenders (see above link for notebook), for in-flight validation to be 
# meaningful during training, our validation set needs not just ground truth, but unseen validation samples 
# to see if predictions for validation users are relevant (to those users). Anyway, break down our val and test 
# sets again to support this strategy
val_src, val_target = python_stratified_split(
    data=val, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )
test_src, test_target = python_stratified_split(
    data=test, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )

In [36]:
print(val.shape, " -> ", val_src.shape, val_target.shape)
print(test.shape, " -> ", test_src.shape, test_target.shape)

(272, 4)  ->  (268, 4) (4, 4)
(733, 4)  ->  (727, 4) (6, 4)


In [37]:
#to use standard names across the analysis 
header = {
        "col_user": "user_id",
        "col_item": "item_id",
        "col_rating": "rating",
        # Unclear why this doesn't also eat a timestamp, but many of the functions that split temporally use, fortunately 
        # the column 'timestamp' (i.e. DEFAULT_TIMESTAMP_COL='timestamp') so I think we're fine. 
        # "col_timestamp" : "timestamp"
    }

train_matrix = AffinityMatrix(df=train, **header)
val_matrix = AffinityMatrix(df=val, **header)
val_src_matrix = AffinityMatrix(df=val_src, **header)
val_tgt_matrix = AffinityMatrix(df=val_target, **header)
test_src_matrix = AffinityMatrix(df=test_src, **header)
test_tgt_matrix = AffinityMatrix(df=test_target, **header)

In [38]:
# This generates a sparse array of user vectors, aka user-item matrix
# X[0] is the first user in the list, with entries for all items known when the matrix was constructed in that row
train, _, _ = train_matrix.gen_affinity_matrix()
val, _, _ = val_matrix.gen_affinity_matrix() 
val_src, _, _ = val_src_matrix.gen_affinity_matrix()
val_tgt, _, _ = val_tgt_matrix.gen_affinity_matrix()
test_src, _, _ = test_src_matrix.gen_affinity_matrix()
test_tgt, _, _ = test_src_matrix.gen_affinity_matrix()

In [39]:
from recommenders.utils.python_utils import binarize

train = binarize(train, 3)
val = binarize(train, 3)
val_src = binarize(val_src, 3) 
val_tgt = binarize(val_tgt, 3)
test_src = binarize(test_src, 3)
test_tgt = binarize(test_tgt, 3)

In [43]:
# TODO: Make sure this is reported during training/configuration
sparsity = np.count_nonzero(train)/(train.shape[0]*train.shape[1])*100
print(f"sparsity: {sparsity:.2f}%")

sparsity: 0.16%


## Model Design 

In [44]:
keras.__version__ 

'2.11.0'

In [45]:
from recommenders.models.vae.standard_vae import StandardVAE

In [46]:
model = StandardVAE(
    n_users = train.shape[0], 
    original_dim = train.shape[1],
    intermediate_dim=250, 
    latent_dim=50, 
    n_epochs=1, 
    batch_size=1, 
    k=10, 
    verbose=1, 
    seed=4, 
    save_path="models/svae.hdf5", 
    drop_encoder=0.5, 
    drop_decoder=0.5, 
    annealing=False, 
    beta=1.0) 

2025-07-20 13:19:13.671508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-20 13:19:13.671627: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-20 13:19:13.671663: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-07-20 13:19:13.671685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-07-20 13:19:13.671706: W tensorflow/c

In [47]:
type(train)

numpy.ndarray

In [48]:
model.fit(
    x_train=train, 
    x_valid=val, 
    x_val_tr=val_src, 
    x_val_te=val_tgt, 
    mapper=val_matrix,
    )

/home/grim/.pyenv/versions/recommenders/lib/python3.9/site-packages/recommenders/models/vae/standard_vae.py:402: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.model.fit_generator(


TypeError: in user code:

    File "/home/grim/.pyenv/versions/recommenders/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/grim/.pyenv/versions/recommenders/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/grim/.pyenv/versions/recommenders/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/grim/.pyenv/versions/recommenders/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/grim/.pyenv/versions/recommenders/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/home/grim/.pyenv/versions/recommenders/l

Managing text-based reviews at this scale could be a challenge, and I'd like to steer clear of LLMs for this effort. We could do an embedding on the review and use that for similarity, but we have pretty rich item data. Perhaps let's ignore the collaborative aspect here and build a shopping interface that: 
- surfaces the most popular items, and encourages you to add items to your shopping cart for a big discount/promo
- based on clicks and cart items, improves the recommendations and surfaces new products

We can use an autoencoder to accept a sparse matrix of users and items, learn to reproduce that matrix, and in so doing support prediction on missing values. However, this matrix is of size users x items, which here is 1.8e7 x 1.6e6 = 28,125,000,000 KB (best-case, higher if stored as np floats) ~= 26 TB !! WTF. 
- In the standard VAE example (https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) the clicks are turned into a histogram for each user ... so we have n_user vector of length n_items... then I guess each of these is a training sample. The VAE presumably learns, given a sparse user vector, to predict every rating. This takes the complexity down and gives us a training set we can iterate over. 

Let's avoid any distributional pressure (present in VAE, SVAE, disentangled VAE) and go for a basic autoencoder using the strategy laid out above, i.e.

## Autoencoder Prototype

In [15]:
import os
import math 
import torch 
import pandas as pd 
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
import sys
sys.path.append('..')
from sparse import AffinityMatrix

In [5]:
class Autoencoder(nn.Module):
    """
    Autoencoder

    NOTE: with cues from https://www.geeksforgeeks.org/deep-learning/implementing-an-autoencoder-in-pytorch/
    """

    def __init__(self, dims=1000):
        """
        Initialize a new object given an item count 
        """
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(dims, 500),
            nn.ReLU(), 
            nn.Linear(500, 75),
            nn.ReLU(), 
        )
        self.decoder = nn.Sequential(
            nn.Linear(75, 500),
            nn.ReLU(), 
            nn.Linear(500, dims),
            nn.ReLU(), 
        )

    def forward(self, x):
        """
        Implement our forward pass 
        """
        h = self.encoder(x) 
        r = self.decoder(h)

        return r

In [ ]:
class DeepCartDataset(torch.utils.data.Dataset): 
    """
    Custom pytorch-compatible dataset. Adapted from 
    https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
    """
    def __init__(self, users, reviews): 
        """
        Initialize a new instance

        Oof. The ideal pattern here is for the dataset to be blissfully ignorant of our split strategy and just 
        make a dataset available to its client based on the raw data passed. However the split strategy is rather intricate below ... 
        note the five splits. Can we easily raise that up to a higher level? The refactoring might not be trivial and it may result in 
        residue of this split strategy bleeding over to the other models -- what's common and what's not? 

        common: 
        - train, val, test split
        - val test and test test bonus splits - for all validation stages we need to check performance, however non-NN techniques 
          really only need a test split, right? if we do train holdout for validation, we essentially have three test sets (train-test, val, test)
        - a need to operate on the same validation or at least test data, lest the comparison be biased by the selection method each model applies

        unique
        - logic to prune reviews < 3.5 -- we don't do this in cfnn, and naive doesn't care (predicts highest review in the matrix), if this is 
          done during training, it will also need to be done during inference
        - need for a pytorch-style dataset ... the naive method is doing a O(n) search, the cfnn needs dataframes -- while refactoring is 
          possible, why understake the risk it will be a disjoint and inelegant fit? 
        - the VAE implementation wants all train and val, but doesn't require a test dataset. we ou


        we could: 
        - pass train and val, hold test out
        - pass test to predict function, which we need for the demo anyway
        - keep the pytorch dataset unique to the pytorch-compatible class... doesn't make sense to try and foist on other algos... we are 
        doing this in the wrong order, filtering and then splitting... we need to outsource the splitting and then do the filtering inside each 
        model 

        right now this is speculation, just get something working! we can figure out how to streamline after -- oh, but we need a dataset 
        implementation
        """
        self.users = users 
        self.reviews = reviews 
        self.matrix = 

    def build_affinity_matrices(): 
        """
        
        """
        oof

    def split(users, reviews, items):
        """
        Generate splits 
        """
        print(f"Full user-item matrix is {len(users) * len(items)}")

        # We are trying to teach the model what a good interaction is like, and we'll 
        # ultimately be interested only in whether to recommend an item or not ... 
        # low reviews are not something we want the model suggesting... 
        reviews_low = reviews[reviews.rating < 3]
        reviews = reviews[reviews.rating >= 3]

        # NOTE: Strategy adapted from tutorials available in the Recommenders project, see 
        # https://github.com/recommenders-team/recommenders/tree/main
        # Split along user boundaries to ensure no leakage of preference between train and test
        train_users, test_users, val_users = python_random_split(users, [.9, .05, .05])
        print(train_users.shape, test_users.shape, val_users.shape)

        train = reviews[reviews.user_id.isin(train_users.user_id)]
        val = reviews[reviews.user_id.isin(val_users.user_id)]
        test = reviews[reviews.user_id.isin(test_users.user_id)]
        print(train.shape, val.shape, test.shape)
        
        # Technique from Recommenders (see https://github.com/recommenders-team/recommenders/blob/45e1b215a35e69b92390e16eb818d4528d0a33a2/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) 
        # to improve utility of validation set during training - only allow items in
        # the validation set that are also present in the train set
        val = val[val.item_id.isin(train.item_id.unique())]
        print(val.shape)

        # Another technique employed in Recommenders (see above link for notebook), for in-flight validation to be 
        # meaningful during training, our validation set needs not just ground truth, but unseen validation samples 
        # to see if predictions for validation users are relevant (to those users). Anyway, break down our val and test 
        # sets again to support this strategy
        val_src, val_target = python_stratified_split(
            data=val, 
            ratio=0.8, 
            filter_by="item", 
            col_user="user_id", 
            col_item="item_id"
            )
        test_src, test_target = python_stratified_split(
            data=test, 
            ratio=0.8, 
            filter_by="item", 
            col_user="user_id", 
            col_item="item_id"
            )
        
        print(val.shape, " -> ", val_src.shape, val_target.shape)
        print(test.shape, " -> ", test_src.shape, test_target.shape)

        train_matrix = AffinityMatrix(df=train, **header)
        val_matrix = AffinityMatrix(df=val, **header)
        val_src_matrix = AffinityMatrix(df=val_src, **header)
        val_tgt_matrix = AffinityMatrix(df=val_target, **header)
        test_src_matrix = AffinityMatrix(df=test_src, **header)
        test_tgt_matrix = AffinityMatrix(df=test_target, **header)

        # This generates a sparse array of user vectors, aka user-item matrix
        # X[0] is the first user in the list, with entries for all items known when the matrix was constructed in that row
        train, _, _ = train_matrix.gen_affinity_matrix()
        val, _, _ = val_matrix.gen_affinity_matrix() 
        val_src, _, _ = val_src_matrix.gen_affinity_matrix()
        val_tgt, _, _ = val_tgt_matrix.gen_affinity_matrix()
        test_src, _, _ = test_src_matrix.gen_affinity_matrix()
        test_tgt, _, _ = test_src_matrix.gen_affinity_matrix()    

        train = binarize(train, 3)
        val = binarize(train, 3)
        val_src = binarize(val_src, 3) 
        val_tgt = binarize(val_tgt, 3)
        test_src = binarize(test_src, 3)
        test_tgt = binarize(test_tgt, 3)

        sparsity = np.count_nonzero(train)/(train.shape[0]*train.shape[1])*100
        print(f"sparsity: {sparsity:.2f}%")
    def __len__(self): 
        """
        Retrieve length of the dataset
        """
        return len(self.img_labels) 
    
    def __getitem__(self, idx): 
        """
        Retrieve an item at the provided index
        """
        #TODO: implement
        pass

In [7]:
def get_data_loader(batch_size=5, shuffle=True): 
    """
    Retrieve a pytorch-style dataloader 
    """

    #TODO: implement
    #transform = transforms.Compose([
    #     transforms.ConvertImageDtype(torch.float),
    #     transforms.Normalize(mean=[0.5], std=[0.5])
    #])

    #data = DeepCartDataset(transform=transform)
    #loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    
    #return loader
    pass

In [8]:
def train(loader, model, loss_interval=20, epochs=2, lr=0.01, momentum=0.9):
    """
    Train the model with the provided dataset

    NOTE: this is a similar training loop as we used for our vision model in the 
    the vision project, forward pass
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    train_loss = []

    tqdm.write(f"Starting training run...")    
    # TODO: configure WandB
    # see https://docs.wandb.ai/guides/integrations/pytorch/
    config = {}
    run = wandb.init(config=config) 

    model.train()
    model = model.to(device)
    
    loss_fn = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(loader):

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # collect metrics
            running_loss += loss.item()

            if (i % loss_interval) == (loss_interval - 1): 
                train_loss.append(running_loss / loss_interval)
                tqdm.write(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / loss_interval:.3f}")
                running_loss = 0 
    
    tqdm.write("Training complete!") 

    return train_loss 